In [1]:
import pandas as pd
import numpy as np
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

bids = pd.read_csv('bids.csv')
traindata = pd.read_csv('train.csv')
testdata = pd.read_csv('test.csv')
trainbids = bids[bids['bidder_id'].isin(traindata.bidder_id)]
testbids = bids[bids['bidder_id'].isin(testdata.bidder_id)]

In [2]:
######################################################
#Every bidder have
#Country_id (not important - assumed )
#No_of_countries-------------------------------------
#No_of_devices---------------------------------------
#Merchandise1
#---
#Merchandisen
#No_of_merchandise-----------------------------------
#Time(max-min)
#No_of_auction---------------------------------------
#Url-------------------------------------------------
######################################################

traindata = traindata.drop(traindata[~traindata.bidder_id.isin(bids.bidder_id)].index)
# testdata = testdata.drop(testdata[~testdata.bidder_id.isin(bids.bidder_id)].index)
# print(traindata.shape, testdata.shape, trainbids.shape, testbids.shape, bids.shape)
n = trainbids.groupby(['bidder_id'])['country', 'device', 'merchandise', 'auction', 'url'].nunique()
nt = testbids.groupby(['bidder_id'])['country', 'device', 'merchandise', 'auction', 'url'].nunique()
# a = '0318e0a173f7c65db40116b903884c854x258'
# print(trainbids['url'].nunique(), trainbids.shape, trainbids['bidder_id'].nunique())
# print(n.head(20))
# print(traindata['payment_account'].nunique(), traindata['address'].nunique(), traindata.shape)
c = []
d = []
a = []
m = []
u = []
for i in testdata['bidder_id']:
    if i in n.index:
        c.append(n['country'][i])
        m.append(n['merchandise'][i])
        a.append(n['auction'][i])
        d.append(n['device'][i])
        u.append(n['url'][i])
    else:
        a.append(0)
        d.append(0)
        c.append(0)
        m.append(0)
        u.append(0)
testdata['countries'] = c
testdata['devices'] = d
testdata['auctions'] = a
testdata['merchandises'] = m
testdata['urls'] = u
traindata['countries'] = [n['country'][i] for i in traindata['bidder_id']]
traindata['auctions'] = [n['auction'][i] for i in traindata['bidder_id']]
traindata['merchandises'] = [n['merchandise'][i] for i in traindata['bidder_id']]
traindata['devices'] = [n['device'][i] for i in traindata['bidder_id']]
traindata['urls'] = [n['url'][i] for i in traindata['bidder_id']]
# testdata['countries'] = [nt['country'][i] for i in testdata['bidder_id']]
# testdata['auctions'] = [nt['auction'][i] for i in testdata['bidder_id']]
# testdata['merchandises'] = [nt['merchandise'][i] for i in testdata['bidder_id']]
# testdata['devices'] = [nt['device'][i] for i in testdata['bidder_id']]
# testdata['urls'] = [nt['url'][i] for i in testdata['bidder_id']]
traindata = traindata.drop(['payment_account', 'address'], axis=1)
X_test = testdata.drop(['bidder_id', 'payment_account', 'address'], axis=1)
X_train = traindata.drop(['bidder_id', 'outcome'], axis=1)
Y_train = traindata['outcome']
traindata.head(100)

,bidder_id,outcome,countries,auctions,merchandises,devices,urls
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,0.0,6,18,1,14,1
1,624f258b49e77713fc34034560f93fb3hu3jo,0.0,1,1,1,2,2
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,0.0,1,4,1,2,2
3,4bee9aba2abda51bf43d639013d6efe12iycd,0.0,1,1,1,1,1
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,0.0,2,23,1,53,91
5,7eaefc97fbf6af12e930528151f86eb91bafh,0.0,1,1,1,1,1
6,25558d24bca82beef0f9db4ba1fe2045ynnvq,0.0,1,8,1,2,8
7,88ae7a35e374a6fddd079ebb28c822eeohwse,0.0,3,10,1,3,1
8,57db69e32163f3e486dc6ef7d615aa12usje6,0.0,1,2,1,2,2
9,d1be739798ba0745a1fd72ac918a9f1929hei,0.0,5,10,1,11,7


In [3]:
logreg = LogisticRegression(solver="lbfgs")
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train,Y_train)*100,2)
coeff_df = pd.DataFrame(X_train.columns)
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])
coeff_df.sort_values(by='Correlation', ascending=False)
print(acc_log)

94.71


In [14]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

95.61

In [16]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

/home/krullun/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


98.94

In [18]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

91.28

In [20]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

/home/krullun/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


36.34

In [22]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

/home/krullun/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


88.41

In [24]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

99.6

In [11]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

99.6

In [25]:
submission = pd.DataFrame({
        "bidder_id": testdata["bidder_id"],
        "prediction": Y_pred
    })
submission.to_csv('submission.csv', index=False)

import subprocess
#subprocess.check_output(['ls','-l']) #all that is technically needed...
print(subprocess.check_output(['kaggle','competitions','submit', '-c', \
                               'facebook-recruiting-iv-human-or-bot', '-f', 'submission.csv', '-m', '"Message"']))

b'Successfully submitted to Facebook Recruiting IV: Human or Robot?'


In [12]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,99.60
7,Decision Tree,99.60
0,Support Vector Machines,98.94
1,KNN,95.61
2,Logistic Regression,94.71
6,Stochastic Gradient Decent,94.25
4,Naive Bayes,91.28
5,Perceptron,36.34


In [13]:
# msk = np.random.rand(len(bids)) < 0.001
# smplbid = bids[msk]
# smplbid.to_csv('sample_bids.csv', index=False)
# smpbid.head()
# print(len(smplbid))